In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn


sys.path.append("../../..")
from batchflow import *
from batchflow.opensets import MNIST
from batchflow.models.torch import *
from batchflow.models.torch.layers import *

# Setup

In [2]:
mnist = MNIST(batch_class=ImagesBatch)

if __name__ == '__main__':
    MICROBATCH = None
    DEVICE = None

print('\nMicrobatching is: {}'.format(MICROBATCH))
print('\nDevice is: {}'.format(DEVICE))    


Microbatching is: None

Device is: None


In [3]:
IMAGE_SHAPE = (1, 28, 28)
N_ITERS = 10
BATCH_SIZE = 16

def get_classification_config(model_class, config):
    default_config = {
        'inputs/images/shape': IMAGE_SHAPE, # can be commented
        'inputs/labels/classes': 10,
        'initial_block/inputs': 'images',   # can be commented
        'loss': 'ce',
        'microbatch': MICROBATCH,
        'device': DEVICE,
    }

    pipeline_config = {
        'model': model_class,
        'model_config': {**default_config, **config},
        'feed_dict': {'images': B('images'),
                      'labels': B('labels')},
    }
    return pipeline_config

def get_segmentation_config(model_class, config):
    default_config = {
        'inputs/images/shape': IMAGE_SHAPE, # can be commented
        'inputs/masks/shape': IMAGE_SHAPE,  # can be commented
        'initial_block/inputs': 'images',   # can be commented
        'loss': 'mse',
        'microbatch': MICROBATCH,
        'device': DEVICE,
    }
    
    pipeline_config = {
        'model': model_class,
        'model_config': {**default_config, **config},
        'feed_dict': {'images': B('images'),
                      'masks': B('images')},
    }
    return pipeline_config

In [4]:
def get_pipeline(pipeline_config):
    """ Pipeline config must contain 'model', 'model_config', 'feed_dict' keys. """
    vals = pipeline_config['feed_dict'].values()

    pipeline = (Pipeline(config=pipeline_config)
                .init_variable('loss_history', [])
                .multiply(multiplier=1/255., preserve_type=False)
                .to_array(channels='first', dtype='float32')
                .init_model('dynamic', C('model'),
                            'MODEL', config=C('model_config'))
                .train_model('MODEL',
#                              *vals,
#                              feed_dict=C('feed_dict'),
                             **pipeline_config['feed_dict'],
                             fetches='loss',
                             save_to=V('loss_history', mode='a'))
                )
    return pipeline

In [5]:
def run(task, model_class, config, description, batch_size=BATCH_SIZE, n_iters=N_ITERS):
    if task.startswith('c'):
        pipeline_config = get_classification_config(model_class, config)
    elif task.startswith('s'):
        pipeline_config = get_segmentation_config(model_class, config)
        
    train_pipeline = get_pipeline(pipeline_config) << mnist.train
    _ = train_pipeline.run(batch_size, n_iters=n_iters, bar=True,
                           bar_desc=W(V('loss_history')[-1].format('Loss is {:7.7}')))
    
    print('{} {} is done'.format(task, description))
    return train_pipeline

In [6]:
def test(pipeline):
    test_pipeline = (mnist.test.p
                    .import_model('MODEL', pipeline)
                    .init_variable('predictions')
                    .init_variable('metrics', init_on_each_run=None) 
                    .to_array(channels='first', dtype='float32')
                    .predict_model('MODEL',
#                                    B.images,
                                   feed_dict={'images': B.images, 'targets': B.labels},
                                   fetches='predictions', save_to=V('predictions'))
                    .gather_metrics('class', targets=B.labels, predictions=V('predictions'),
                                    fmt='logits', axis=-1, save_to=V('metrics', mode='w'))
                    .run(64, shuffle=True, n_epochs=1, drop_last=False, bar=False)
    )
    
    metrics = test_pipeline.get_variable('metrics')
    print('Accuracy is: {}'.format(metrics.evaluate('accuracy')))

# Classification

In [7]:
config = {
    'initial_block': {'layout': 'fa'*2,
                      'units': [64, 128],},
    'body': {'layout': 'fa'*2,
             'units': [256, 512]},
    'head': {'layout': 'faf',
             'units': [600, 10]},
}

ppl = run('classification', TorchModel, config, 'simple fc', n_iters=50, batch_size=64)
test(ppl)

Loss is 2.312046: 100%|██████████| 50/50 [00:03<00:00, 13.30it/s]


classification simple fc is done
Accuracy is: 0.1875


In [8]:
config = {
    'body/encoder/num_stages': 5,
    'head': {'layout': 'f'}
}

ppl = run('classification', Encoder, config, 'encoder')

Loss is 2.568651: 100%|██████████| 10/10 [00:00<00:00, 14.95it/s]

classification encoder is done


In [9]:
config = {
    'initial_block': {'layout': 'fafaf', 'units': [128, 256, 10]},
    'order': ['initial_block', ('ib_2', 'initial_block', TorchModel.initial_block)],
    'loss': ['ce', 'ce'],
    'decay': 'exp',
    'n_iters': 25,
    'train_steps': {'ts_1': {}, 'ts_2': {}},
}

ppl = run('classification', TorchModel, config, 'train steps and order')

Loss is 4.621878: 100%|██████████| 10/10 [00:00<00:00, 14.85it/s]

classification train steps and order is done


In [10]:
config = {
    'initial_block/inputs': ['images', 'images'],
    # note that we can't directly assign this module to `initial_block`
    'initial_block/module': Combine(op='+'),
    'body/encoder': {'num_stages': 5},
    'head': {'layout': 'faf', 'units': [50, 10]}
}

ppl = run('classification', Encoder, config, 'duo input')

Loss is 2.304632: 100%|██████████| 10/10 [00:00<00:00, 14.62it/s]

classification duo input is done


In [11]:
config = {
    'initial_block/filters': 6,
    'body/encoder/blocks/n_reps': [1, 1, 2, 1],
    'body/encoder/blocks/bottleneck': False,
    'body/encoder/blocks/se': False,
}

ppl = run('classification', ResNet, config, 'resnet with config')

Loss is 4.394666: 100%|██████████| 10/10 [00:06<00:00,  1.75it/s]

classification resnet with config is done


In [12]:
config = {
}

# ppl = run('classification', VGG7, config, 'vgg7')
ppl = run('classification', ResNet18, config, 'resnet18')
# ppl = run('classification', SEResNeXt18, config, 'SE-resneXt18')
# ppl = run('classification', DenseNet121, config, 'DenseNet121')

Loss is 2.774392: 100%|██████████| 10/10 [00:11<00:00,  1.07s/it]

classification resnet18 is done


In [13]:
# reusing encoder from model from the previous cell
encoder = ppl.m('MODEL').get('encoder', from_start=True, wrap=True)
config = {
    'initial_block': encoder,
    'head' : {'layout': 'Vf'},
}

ppl = run('classification', TorchModel, config, 'reused encoder')

Loss is 3.577238: 100%|██████████| 10/10 [00:10<00:00,  1.10s/it]

classification reused encoder is done


In [14]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = torch.nn.Identity()

config = {
    'initial_block': {'layout': 'cna',
                      'filters': 3},
    'body': resnet18,
    'head': {'layout': 'Dnfaf',
             'units': [50, 10],
             'dropout_rate': 0.3,
             'multisample': 0.3},
}

ppl = run('classification', TorchModel, config, 'pretrained resnet')

Loss is 2.303532: 100%|██████████| 10/10 [00:07<00:00,  1.28it/s]

classification pretrained resnet is done


# Segmentation

In [15]:
config = {
    'initial_block': {'layout': 'cna', 'filters': 8},
    'body/decoder/num_stages': 3,
    'body/decoder/factor': [1, 1, 1],
}

ppl = run('segmentation', Decoder, config, 'decoder')

Loss is 0.05024578: 100%|██████████| 10/10 [00:01<00:00,  8.52it/s]

segmentation decoder is done


In [16]:
config = {
    'step_on_each': 1,
    'initial_block': {
        'layout': 'cnaRp cnaRp tna+ tna+ BScna*+ cnac',
        'filters': [16, 32, 32, 16, 'same', 8, 1],
        'transposed_conv': {'kernel_size': 2, 'strides': 2},
        'branch': {'layout': 'ca', 'filters': 'same'}
    },
}

ppl = run('segmentation', TorchModel, config, 'hardcoded unet')

Loss is 0.01167104: 100%|██████████| 10/10 [00:01<00:00,  6.24it/s]

segmentation hardcoded unet is done


In [17]:
config = {
    'body/encoder/num_stages': 2,
    'body/embedding': {'base': ASPP, 'pyramid': (2, 4, 8)},
}

ppl = run('segmentation', EncoderDecoder, config, 'unet-like with ASPP')

Loss is 0.1696641: 100%|██████████| 10/10 [00:01<00:00,  8.16it/s]

segmentation unet-like with ASPP is done


In [18]:
config = {
    'initial_block/filters': 128,
    'body/encoder/num_stages': 3,
    'body/encoder/blocks/filters': 'same*2',
    'body/embedding/filters': 'same',
    'body/decoder/blocks/filters': 'same//2',
}

ppl = run('segmentation', UNet, config, 'unet')
# ppl = run('segmentation', ResUNet, config, 'unet with residual blocks')

Loss is 0.09976862: 100%|██████████| 10/10 [00:02<00:00,  4.78it/s]

segmentation unet is done


In [20]:
config = {
    'initial_block/filters': 16,
    'body/encoder/num_stages': 2,
    'body/embedding/filters': 6,
    'body/decoder/blocks/filters': 6,
}

ppl = run('segmentation', DenseUNet, config, 'unet with dense blocks')

Loss is 0.04701341: 100%|██████████| 10/10 [00:05<00:00,  2.06it/s]

segmentation unet with dense blocks is done


In [21]:
config = {
    'body/encoder/base_model': ResNet18,
    'body/encoder/base_model_kwargs/blocks/filters': 7, 
    'body/decoder/blocks/filters': 'same//4'
}

ppl = run('segmentation', EncoderDecoder, config, 'encoder-decoder with resnet18 backbone')

Loss is 0.1330853: 100%|██████████| 10/10 [00:02<00:00,  4.72it/s]

segmentation encoder-decoder with resnet18 backbone is done


In [22]:
ppl.m('MODEL').info


##### Config:
{'benchmark': True,
 'body': {'decoder': {'blocks': {'base': <class 'batchflow.models.torch.blocks.DefaultBlock'>,
                                 'filters': 'same//4'},
                      'combine': {'op': 'concat'},
                      'factor': None,
                      'num_stages': None,
                      'order': ['upsampling', 'block', 'combine'],
                      'skip': True,
                      'upsample': {'layout': 'tna'}},
          'embedding': {'base': <class 'batchflow.models.torch.blocks.DefaultBlock'>},
          'encoder': {'base_model': <class 'batchflow.models.torch.resnet.ResNet18'>,
                      'base_model_kwargs': {'blocks': {'filters': 7}},
                      'blocks': {'base': <class 'batchflow.models.torch.blocks.DefaultBlock'>},
                      'downsample': {'layout': 'p',
                                     'pool_size': 2,
                                     'pool_strides': 2},
                      'n

In [ ]:
ppl.m('MODEL').model